In [1]:
# 1. 匯入函式庫
!pip install numpy
import numpy as np

# 2. 定義重力學常數 (mGal, meter, degree)
# 國際重力公式常數 (IGF/WGS84 標準)
G_E = 978032.7       # 赤道重力 (mGal) - 使用標準值
k = 0.0053024        # 重力扁率常數
k_prime = 0.0000059  # 另一常數項

# 修正項常數
FAA_GRADIENT = 0.3086   # 自由空氣梯度 (mGal/m)
RHO_CRUST = 2.67        # 陸地 Bouguer 校正密度 (g/cm^3)
RHO_WATER = 1.03        # 海水密度 (g/cm^3)
BOUGUER_CONSTANT = 0.04192 # Bouguer 板校正常數 (mGal/(g/cm^3 * m))

# 3. 輸入題目數據
LATITUDE = 48.1195   # 緯度 (度 N)
H_ELEVATION = 487.9  # 高度 H (m) / 水深 D (m)
G_OBS = 980717.39    # 觀測重力 (mGal)

print("✅ 常數與題目數據載入完成。")

✅ 常數與題目數據載入完成。


In [2]:
# 1. 理論重力公式
def theoretical_gravity(phi_deg):
    """g_theo = G_E * (1 + k * sin^2(phi) - k' * sin^4(phi))"""
    phi_rad = np.deg2rad(phi_deg)
    sin2_phi = np.sin(phi_rad)**2
    sin4_phi = np.sin(phi_rad)**4
    
    g_theo = G_E * (1 + k * sin2_phi - k_prime * sin4_phi)
    return g_theo

# 2. 自由空氣校正 (FAC)
def free_air_correction(H):
    """FAC = 0.3086 * H"""
    return FAA_GRADIENT * H

# 3. Bouguer 板校正 (BC)
def bouguer_correction(H, rho):
    """BC = 0.04192 * rho * H"""
    return BOUGUER_CONSTANT * rho * H

print("✅ 計算函數定義完成。")

✅ 計算函數定義完成。


In [3]:
# --- (a) 陸地站點計算 (Elevation H=487.9 m) ---
H_a = H_ELEVATION
print(f"--- (a) 陸地站點計算 --- (H = {H_a} m)\n")

# i) Theoretical Gravity (g_theo)
g_theo_a = theoretical_gravity(LATITUDE)
print(f"i) Theoretical Gravity (g_theo): {g_theo_a:.3f} mGal")

# ii) Free Air Correction (FAC)
fac_a = free_air_correction(H_a)
print(f"ii) Free Air Correction (FAC): +{fac_a:.3f} mGal")

# iii) Bouguer Correction (BC, rho=2.67 g/cm^3)
bc_a = bouguer_correction(H_a, RHO_CRUST)
print(f"iii) Bouguer Correction (BC): -{bc_a:.3f} mGal")

--- (a) 陸地站點計算 --- (H = 487.9 m)

i) Theoretical Gravity (g_theo): 980905.680 mGal
ii) Free Air Correction (FAC): +150.566 mGal
iii) Bouguer Correction (BC): -54.609 mGal


In [4]:
print(f"--- (a) 陸地站點異常值計算 ---\n")

# iv) Free Air Gravity Anomaly (FAA)
# FAA = g_obs - g_theo + FAC
faa_a = G_OBS - g_theo_a + fac_a
print(f"iv) Free Air Gravity Anomaly (FAA): {faa_a:.3f} mGal")

# v) Bouguer Gravity Anomaly (BA)
# BA = FAA - BC
ba_a = faa_a - bc_a
print(f"v) Bouguer Gravity Anomaly (BA): {ba_a:.3f} mGal")

--- (a) 陸地站點異常值計算 ---

iv) Free Air Gravity Anomaly (FAA): -37.724 mGal
v) Bouguer Gravity Anomaly (BA): -92.333 mGal


In [5]:
# --- (b) 海洋站點異常值計算 (觀測在海平面, 水深 D=487.9 m) ---
D_water = H_ELEVATION # 水深 D = 487.9 m
H_b = 0 # 觀測高度 H = 0
print(f"--- (b) 海洋站點計算 --- (水深 D = {D_water} m, H = {H_b} m)\n")

# Recompute FAA: FAC (H=0) = 0
fac_b = free_air_correction(H_b) 
faa_b = G_OBS - g_theo_a + fac_b 
print(f"i) Free Air Gravity Anomaly (FAA_b): {faa_b:.3f} mGal (FAC=0)")

# 海洋 Bouguer 異常 (BA_ocean)
# BC_ocean = 0.04192 * D * (rho_water - rho_crust)
rho_contrast_b = RHO_WATER - RHO_CRUST # 1.03 - 2.67
bc_ocean = bouguer_correction(D_water, rho_contrast_b)
print(f"ii) 海洋 Bouguer 校正 (BC_ocean, rho_c={rho_contrast_b:.2f}): {bc_ocean:.3f} mGal")

# 布格異常 (BA_b)
# BA_ocean = FAA_b + BC_ocean
ba_ocean_b = faa_b + bc_ocean
print(f"iii) Bouguer Gravity Anomaly (BA_b): {ba_ocean_b:.3f} mGal")

print("\n--- ✅ 作業 8-2 完整計算完成。 ---")

--- (b) 海洋站點計算 --- (水深 D = 487.9 m, H = 0 m)

i) Free Air Gravity Anomaly (FAA_b): -188.290 mGal (FAC=0)
ii) 海洋 Bouguer 校正 (BC_ocean, rho_c=-1.64): -33.543 mGal
iii) Bouguer Gravity Anomaly (BA_b): -221.832 mGal

--- ✅ 作業 8-2 完整計算完成。 ---
